# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [24]:
#WORKING

# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bilibino,68.0546,166.4372,-27.75,100,96,1.07,RU,1699645133
1,1,bubaque,11.2833,-15.8333,29.09,52,94,3.79,GW,1699645133
2,2,port-aux-francais,-49.3500,70.2167,7.17,97,100,16.51,TF,1699645134
3,3,padang,-0.9492,100.3543,24.55,94,100,0.94,ID,1699645134
4,4,st. john's,47.5649,-52.7093,4.43,88,75,10.29,CA,1699644894


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
# Configure the map plot
city_humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["City", "Humidity"],
    alpha=0.4
)

# Display the map
city_humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values

#define ideal weather criteria
min_temp = 24
max_temp =30
max_humidity = 50
max_cloudiness = 25
max_wind_speed = 10

#filter the df
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] >= min_temp) & 
    (city_data_df["Max Temp"] <= max_temp) & 
    (city_data_df["Humidity"] < max_humidity) & 
    (city_data_df["Cloudiness"] < max_cloudiness) & 
    (city_data_df["Wind Speed"] < max_wind_speed)
]

# Drop any rows with null values
ideal_weather_df_cleaned = city_data_df.dropna()

# Display sample data
ideal_weather_df_cleaned.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bilibino,68.0546,166.4372,-27.75,100,96,1.07,RU,1699645133
1,1,bubaque,11.2833,-15.8333,29.09,52,94,3.79,GW,1699645133
2,2,port-aux-francais,-49.3500,70.2167,7.17,97,100,16.51,TF,1699645134
3,3,padang,-0.9492,100.3543,24.55,94,100,0.94,ID,1699645134
4,4,st. john's,47.5649,-52.7093,4.43,88,75,10.29,CA,1699644894


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
78,guerrero negro,MX,27.9769,-114.0611,19,
109,vallegrande,BO,-18.4897,-64.1069,37,
126,vandenberg village,US,34.7083,-120.4677,39,
243,la palma,US,33.8464,-118.0467,27,
274,letlhakeng,BW,-24.0944,25.0298,31,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [43]:
# Set parameters to search for a hotel

radius = 10000
limit = 1
params = {
    'limit':limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["categories"]=f"accommodation.hotel"
    params["filter"] = f"&circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
guerrero negro - nearest hotel: No hotel found
vallegrande - nearest hotel: No hotel found
vandenberg village - nearest hotel: No hotel found
la palma - nearest hotel: No hotel found
letlhakeng - nearest hotel: No hotel found
kuraymah - nearest hotel: No hotel found
atar - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
tindouf - nearest hotel: No hotel found
opuwo - nearest hotel: No hotel found
tidjikja - nearest hotel: No hotel found
umm kaddadah - nearest hotel: No hotel found
maun - nearest hotel: No hotel found
ghat - nearest hotel: No hotel found
ad dabbah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
78,guerrero negro,MX,27.9769,-114.0611,19,No hotel found
109,vallegrande,BO,-18.4897,-64.1069,37,No hotel found
126,vandenberg village,US,34.7083,-120.4677,39,No hotel found
243,la palma,US,33.8464,-118.0467,27,No hotel found
274,letlhakeng,BW,-24.0944,25.0298,31,No hotel found
295,kuraymah,SD,18.5500,31.8500,27,No hotel found
298,atar,MR,20.5169,-13.0499,19,No hotel found
318,taoudenni,ML,22.6783,-3.9836,14,No hotel found
336,tindouf,DZ,27.6711,-8.1474,18,No hotel found
413,opuwo,NaN,-18.0607,13.8400,37,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [41]:
# Configure the map plot
city_hotel_map = hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    tiles='OSM', 
    frame_width=800, 
    frame_height=600, 
    size='Humidity', 
    scale=2, 
    color='City', 
    alpha=.4,
    hover_cols=['City', 'Hotel Name', 'Country', 'Humidity']  # Add 'Hotel Name' and 'Country' to hover tooltip
)

# Display the map
city_hotel_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)